In [28]:
import os

In [29]:
os.getcwd()

'/Users/lorenzo.marinucci96/Documents/GitHub/simplicial_model'

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from library import *
import cvxpy as cp

In [2]:
def opt_lambda(Q):
    """
    Risolve il problema:
        minimize   s.T @ Q @ s - 2 * sum(log(s))
        subject to s > 0

    Args:
        Q: matrice (n x n), simmetrica definita positiva

    Returns:
        s_opt: soluzione ottimale (numpy array)
    """
    n = Q.shape[0]
    s = cp.Variable(n, pos=True)  # impone s_i > 0

    objective = cp.Minimize(cp.quad_form(s, Q) - 2 * cp.sum(cp.log(s)))
    problem = cp.Problem(objective)

    problem.solve()

    if problem.status != cp.OPTIMAL:
        raise RuntimeError(f"Problema non risolto: {problem.status}")

    return s.value


In [3]:
def opt_Theta(H, T, G, verbose=False):
    """
    Risolve il problema:
        min_X trace(X @ H) - logdet(X)
        s.t. X PSD
             X_{ij} = 0  where T_{ij} = 0 (sparsità)
             (X - I)(G - I) = 0
             
    Args:
        H: matrice simmetrica definita positiva (n x n)
        T: matrice di maschera binaria (n x n), T[i,j] == 0 ⇒ X[i,j] == 0
        G: matrice reale (n x n)
        verbose: stampa output del solver
        
    Returns:
        X_opt: soluzione ottima come array NumPy (n x n)
    """
    n = H.shape[0]
    I = np.eye(n)
    mask = 1-T
    
    # Variabile simmetrica
    X = cp.Variable((n, n), symmetric=True)

    # Obiettivo: trace(XH) - logdet(X)
    #penalty = cp.norm1(cp.multiply(mask, X)) # vincolo topologia
    objective = cp.Minimize(cp.trace(X @ H) - cp.log_det(X))

    constraints = []

    # 1. Vincolo PSD
    constraints.append(X >> 0)

     #2. Sparsità da T
    for i in range(n):
        for j in range(n):
           if T[i, j] == 0:
                constraints.append(X[i, j] == 0)

    # 3. (X - I)(G - I) = 0 ⇒ per ogni elemento (riga i, colonna j):
    #     sum_k (X[i,k] - δ_ik)(G[k,j] - δ_kj) = 0
    G_shift = G - I
    for i in range(n):
        for j in range(n):
            expr = sum((X[i, k] - (1.0 if i == k else 0.0)) * G_shift[k, j] for k in range(n))
            constraints.append(expr == 0)

    # Risoluzione
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, verbose=verbose)

    return X.value


In [4]:
def same_sparsity_structure(A, B, threshold=1e-10):
    # Azzeramento sotto soglia
    A_clean = A.copy()
    A_clean[np.abs(A_clean) < threshold] = 0.0

    B_clean = B.copy()
    B_clean[np.abs(B_clean) < threshold] = 0.0

    # Maschere di elementi non nulli
    mask_A = A_clean != 0.0
    mask_B = B_clean != 0.0

    # Confronto posizione per posizione
    same_structure = np.array_equal(mask_A, mask_B)

    return same_structure, mask_A, mask_B

In [5]:
max_dimension = 2 # Dimension of the complex, e.g. 2 -> nodes, egdes, triangles
p_complex = [.3,.6]
max_num_simplices = 60 # Maximum number of simplices
min_num_simplices = 20 # Minimum number of simplices
avg_nodes = 10
variance_complex = 2

In [6]:
valid_data_point = 0
valid_complex = 0
while not valid_complex:
      _, incidence_mats, num_simplices = generate_simplicial_complex(avg_nodes, max_dimension, p_complex, variance_complex)
      if num_simplices < max_num_simplices and num_simplices >= min_num_simplices:
        valid_complex = 1
        print("Valid complex generated/loaded!")
      else:
       print("The generated complex does not respect the imposed constraints... Trying again!")

Complex correctly saved on the current path, veryfing if valid...
Valid complex generated/loaded!


In [7]:
B1=incidence_mats[1]
B1.shape

(12, 20)

In [8]:
B2=incidence_mats[2]
B2.shape

(20, 2)

In [9]:
n_nodes=B1.shape[0]
n_edges=B1.shape[1]
n_triangles=B2.shape[1]

In [10]:
D_V=5*np.eye(n_nodes)
D_E=6*np.eye(n_edges)
D_T=5*np.eye(n_triangles)

In [11]:
row1 = np.hstack([D_V, -B1, np.zeros((n_nodes,n_triangles))])
# Seconda e terza riga di blocchi (definizione generica, possono variare)
row2 = np.hstack([-B1.T, D_E, -B2])
row3 = np.hstack([np.zeros((n_triangles,n_nodes)), -B2.T, D_T])
prec_matrix = np.vstack([row1, row2, row3])
eigvals = np.linalg.eigvalsh(prec_matrix)  # Calcola gli autovalori (più efficiente per matrici simmetriche)
check=np.all(eigvals > 0)
check

True

In [12]:
row1 = np.hstack([D_V, -B1])
row2 = np.hstack([-B1.T, D_E])
mat = np.vstack([row1, row2])
cov_E_d = np.linalg.inv(mat)[n_nodes:,n_nodes:]
prec_E_d = np.linalg.inv(cov_E_d)
prec_E_d[np.abs(prec_E_d) < 1e-10] = 0.0
sparsity,Md,M2=same_sparsity_structure(B1.T @ B1 + np.eye(n_edges), prec_E_d)
sparsity #lower interactions 


True

In [13]:
row1 = np.hstack([D_E, -B2])
row2 = np.hstack([-B2.T, D_T])
mat = np.vstack([row1, row2])
cov_E_u = np.linalg.inv(mat)[:n_edges,:n_edges]
prec_E_u = np.linalg.inv(cov_E_u)
sparsity,Mu,M2=same_sparsity_structure(B2 @ B2.T + np.eye(n_edges), prec_E_u)
prec_E_u[np.abs(prec_E_u) < 1e-10] = 0.0
sparsity #upper interactions

True

In [14]:
cov=np.linalg.inv(prec_matrix)
cov[:n_nodes,-n_triangles:] = np.zeros((n_nodes,n_triangles)) #check condizione indipendenza
cov[-n_triangles:,:n_nodes] = np.zeros((n_triangles,n_nodes))
mu=np.zeros(cov.shape[0])
iterations=50000
X = np.random.multivariate_normal(mu, cov, size=iterations)

In [15]:
X_E=X[:,n_nodes:n_nodes+n_edges]
X_E.shape #osservazioni sugli archi

(50000, 20)

In [16]:
cov_E=cov[n_nodes:n_nodes+n_edges,n_nodes:n_nodes+n_edges]
prec_E=np.linalg.inv(cov_E)
prec_E[np.abs(prec_E) < 1e-10] = 0.0


In [17]:
np.linalg.norm(prec_E-prec_E_d-prec_E_u+D_E) # altro check

3.60448656605019e-15

In [18]:
S = (1/iterations)*X_E.T @ X_E #cov empirica
np.linalg.norm(S-cov_E)

0.01684279854355994

In [19]:
np.linalg.norm(np.linalg.inv(S)-prec_E)

0.5170311361025411

In [20]:
def is_symmetric(A, tol=1e-8):
    return np.allclose(A, A.T, atol=tol)

def is_psd(A):
    return np.all(np.linalg.eigvals(A) >= -1e-8)  # piccolo margine numerico
Theta_u = np.eye(n_edges)
Theta_d = np.eye(n_edges)
Q = (Theta_d + Theta_u - np.eye(n_edges)) * S
print(is_symmetric(Q))
print(is_psd(Q))

True
True


In [25]:
class Inference:
    def __init__(
            self,
            S, 
            MAX_ITER,
            inc_mats
    ):
        self.S = S
        self.MAX_ITER = MAX_ITER
        self.B1 = inc_mats[1]
        self.B2 = inc_mats[2]

        self.n_edges = S.shape[0]

        self.mask_d = (self.B1.T @ self.B1 != 0.0).astype(int)
        self.mask_u = (self.B2 @ self.B2.T != 0.0).astype(int)

        for i in range(self.n_edges):
            self.mask_d[i,i] = 1
            self.mask_u[i,i] = 1 
    
        # Build problems (they will be compiled at first solving)
        self._opt_lambda_build()
        self._opt_Theta_build()


    def _initialization(
            self
    ):
        '''
        Initialize the variables in the optimization problem
        '''

        l = np.sqrt(np.diag(self.S))
        Theta_u = np.eye(self.n_edges)
        Theta_d = np.eye(self.n_edges)

        return l, Theta_d, Theta_u

    #---------------------------#
    #       LAMBDA STEP
    #---------------------------#

    def _opt_lambda_build(
            self
    ):
        """
        Costruisce il problema:
            minimize   l.T @ Q @ l - 2 * sum(log(s))
            subject to s > 0
        """
        self.Q_param = cp.Parameter((self.n_edges, self.n_edges), PSD=True)
        self.l = cp.Variable(self.n_edges, pos=True)  # impone s_i > 0

        objective = cp.Minimize(cp.quad_form(self.l, self.Q_param) - 2 * cp.sum(cp.log(self.l)))
        self.problem_lambda = cp.Problem(objective)
    
    def _opt_lambda_solve(
            self,
            Q_value,
    ):
        self.Q_param.value = Q_value
        self.problem_lambda.solve(solver=cp.SCS) 

        return self.l.value 
    
    #---------------------------#
    #        THETA STEP
    #---------------------------#

    def _opt_Theta_build(
            self,
    ):
        """
        Costruisce il problema:
            min_X trace(X @ H) - logdet(X)
            s.t. X PSD
                X_{ij} = 0  where T_{ij} = 0 (sparsità)
                (X - I)(G - I) = 0
        """

        self.H_param = cp.Parameter((self.n_edges, self.n_edges))
        self.T_param = cp.Parameter((self.n_edges, self.n_edges))
        self.G_param = cp.Parameter((self.n_edges, self.n_edges))


        # Variabile simmetrica
        self.X = cp.Variable((self.n_edges, self.n_edges), PSD=True)

        # Obiettivo: trace(XH) - logdet(X)
        #penalty = cp.norm1(cp.multiply(mask, X)) # vincolo topologia
        objective = (
            cp.Minimize(cp.trace(self.X @ self.H_param) - 
                        cp.log_det(self.X))
        )

        constraints = []

        # 1. Vincolo PSD
        constraints.append(self.X >> 0)

        #2. Sparsità da T
        for i in range(self.n_edges):
            for j in range(self.n_edges):
               constraints.append(self.X[i, j]*(1-self.T_param[i, j]) == 0)

        # 3. (X - I)(G - I) = 0 ⇒ per ogni elemento (riga i, colonna j):
        #     sum_k (X[i,k] - δ_ik)(G[k,j] - δ_kj) = 0
        G_shift = self.Q_param - np.eye(self.n_edges)
        for i in range(self.n_edges):
            for j in range(self.n_edges):
                expr = sum((self.X[i, k] - (1.0 if i == k else 0.0)) * G_shift[k, j] for k in range(self.n_edges))
                constraints.append(expr == 0)

        self.problem_theta = cp.Problem(objective, constraints)

    def _opt_Theta_solve(
            self,
            H_value,
            T_value,
            G_value,
            solver = cp.SCS,
            verbosity = 0
    ):    
        self.H_param.value = H_value
        self.T_param.value = T_value
        self.G_param.value = G_value

        self.problem_theta.solve(solver = solver, verbosity = verbosity) 

        return self.X.value 
    
    #----------------------------------------------------------#
    #        TWO-STEPS ALTERNATED OPTIMIZATION PIPELINE
    #----------------------------------------------------------#

    def _fit(
            self
    ):
        # Initialization
        l, Theta_d, Theta_u = self._initialization()

        for _ in range(self.MAX_ITER):
            print(_)
            Q = (Theta_d + Theta_u - np.eye(n_edges)) * S
            #Q = (Q + Q.T)/2

            l = np.copy(self._opt_lambda_solve(Q))
            H = np.diag(l) @ S @ np.diag(l)

            Theta_d = np.copy(self._opt_Theta_solve(H, self.mask_d, Theta_u))
            Theta_u = np.copy(self._opt_Theta_solve(H, self.mask_u, Theta_d))

        return Theta_d, Theta_u, l , Q

In [26]:
infer=Inference(S,100,incidence_mats)

In [27]:
Theta_d, Theta_u, l , Q = infer._fit()

0


/Users/lorenzo.marinucci96/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:226: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(DPP_ERROR_MSG)


ValueError: Error parsing inputs


In [53]:
Q

array([[ 0.17822235,  0.        ,  0.        ,  0.        , -0.        ,
        -0.        ,  0.        , -0.        , -0.        , -0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.18441019,  0.        ,  0.        , -0.        ,
        -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
         0.        ,  0.        , -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.18485221,  0.        , -0.        ,
        -0.        , -0.        , -0.        ,  0.        , -0.        ,
         0.        , -0.        , -0.        ,  0.        , -0.        ,
         0.        ,  0.        , -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.17937366, -0.        ,
         0.        , -0.        ,  0.        ,  0.        ,  0.        ,
         0.   

In [42]:
def inference(S, incidence_mats, it=100):

    B1 = incidence_mats[1]
    B2 = incidence_mats[2]
    n_edges = S.shape[0]
    mask_d = (B1.T @ B1 != 0.0).astype(int)
    mask_u = (B2 @ B2.T != 0.0).astype(int)
    for i in range(n_edges):
       mask_d[i,i] = 1
       mask_u[i,i] = 1 

    ## Inizializzazione
    l = np.sqrt(np.diag(S))
    Theta_u = np.eye(n_edges)
    #Theta_u[mask_u.astype(bool) & (~np.eye(n_edges, dtype=bool))] = eps
    Theta_d = np.eye(n_edges)
    
    #algoritmo alterato
    for i in range(it):
        Q = (Theta_d + Theta_u - np.eye(n_edges)) * S
        Q = (Q + Q.T)/2
        l = opt_lambda(Q) #ottimizza l
        H = np.diag(l)@S@np.diag(l)  
        Theta_d = opt_Theta(H, mask_d, Theta_u) #ottimizza Theta_d
        Theta_u = opt_Theta(H, mask_u, Theta_d) #ottimizza Theta_u
        print("Iteration: "+str(i+1)+"/100")
        clear_output(wait=True)
    return Theta_d,Theta_u,l

In [241]:
Theta_d,Theta_u,l=inference(S, incidence_mats, it=100)

Iteration: 100/100


In [243]:
l

array([2.22220976, 2.2856553 , 2.2714539 , 2.35802515, 2.29126236,
       2.29880009, 2.26474273, 2.33311151, 2.31134242, 2.22865006,
       2.31769301, 2.3051263 , 2.3039776 , 2.27700471, 2.31954482,
       2.31411953, 2.32368972, 2.35468169, 2.36649741, 2.3576663 ,
       2.35639003, 2.34859223, 2.36553266, 2.34849524, 2.36804459,
       2.27283094])

In [242]:
Theta_d

array([[ 1.00616208e+00, -3.57090239e-02, -2.39781127e-03,
        -3.64955651e-02, -7.48435458e-04,  3.59727907e-03,
         3.17520466e-02, -3.47305065e-04,  3.87760575e-02,
         1.24638302e-03,  3.73083487e-02, -6.96242915e-03,
        -2.62420410e-09,  2.12723708e-09,  2.64923938e-09,
        -9.78622076e-09,  2.18030919e-09,  7.57897348e-09,
         1.24733495e-08, -1.05558512e-08, -4.03699742e-09,
        -9.31802239e-09, -5.43289238e-09, -1.20265391e-08,
        -2.07321040e-08,  2.02056941e-09],
       [-3.57090239e-02,  1.00363592e+00,  4.35014388e-03,
        -3.79912139e-02,  4.19227704e-04, -3.62629105e-02,
         9.62579941e-10,  1.04172977e-08,  8.71072771e-09,
         3.86524607e-09,  6.39203074e-09,  6.59961972e-09,
         1.56783551e-08,  6.74800014e-09,  1.22539338e-08,
        -3.24464537e-04,  3.61687622e-03, -1.22504472e-08,
        -1.02158275e-08,  1.30964218e-08,  2.49175046e-08,
         1.28818782e-08,  1.79045650e-08,  8.15481216e-09,
        -2.24

In [247]:
np.round(prec_E_u,4)

array([[ 5.4,  0. ,  0.2,  0. ,  0.2,  0.2,  0. , -0.2,  0. , -0.2,  0. ,
        -0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  5.6,  0.2,  0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -0.2, -0.2,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0.2,  0.2,  5.6,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  6. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0.2,  0.2,  0. ,  0. ,  5.6,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0.2,  0. ,  0. ,  0. ,  0. ,  5.8,  0. ,  0. ,  0. ,  0. ,  0. ,
         0.2,  0. ,  

In [249]:
np.round(Theta_u*6,5)

array([[ 6., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.],
       [-0.,  6., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.],
       [ 0., -0.,  6., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.],
       [-0.,  0., -0.,  6., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.],
       [ 0., -0.,  0., -0.,  6., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.],
       [-0.,  0., -0.,  0., -0.,  6., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.],
       [ 0., -0.,  0., -0.,  0., -0.,  6.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0

In [155]:
prec_E

array([[ 5.6, -0.2, -0.2, -0.2,  0.2,  0.2,  0.2,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-0.2,  5.6, -0.2, -0.2,  0. ,  0. ,  0. ,  0.2,  0.2,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-0.2, -0.2,  5.4,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.2,
         0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-0.2, -0.2,  0. ,  5.4,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0.2,  0.2,  0. ,  0. ,  0. ],
       [ 0.2,  0. ,  0. ,  0. ,  5.6, -0.2, -0.2, -0.2,  0. ,  0. ,  0. ,
         0. ,  0.2,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.2,  0. ,  0. ,  0. , -0.2,  5.6, -0.2,  0. , -0.2,  0. ,  0. ,
         0. ,  0. , -0.2,  0. ,  0. , -0.2,  0. ,  0.2],
       [ 0.2,  0. ,  0. ,  0. , -0.2, -0.2,  5.6,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -0.2,  0. , -0.2,  0. ],
       [ 0. ,  0.2,  0. ,  0. , -0.2,  0. ,  0. ,  5.4,  0. ,  0. ,  0. ,
         

In [246]:
np.round(np.diag(l) @ Theta_u @ np.diag(l),4)- np.round(prec_E_u,4)

array([[-0.4618, -0.    , -0.2   , -0.    , -0.2   , -0.2   ,  0.    ,
         0.2   , -0.    ,  0.2   , -0.    ,  0.2   , -0.    ,  0.    ,
         0.    , -0.    , -0.    , -0.    , -0.    ,  0.    ,  0.    ,
        -0.    , -0.    ,  0.    ,  0.    , -0.    ],
       [-0.    , -0.3758, -0.2   ,  0.    , -0.2   ,  0.    , -0.    ,
        -0.    ,  0.    , -0.    ,  0.    , -0.    ,  0.    , -0.    ,
        -0.    ,  0.2   ,  0.2   ,  0.    ,  0.    , -0.    , -0.    ,
         0.    ,  0.    , -0.    , -0.    ,  0.    ],
       [-0.2   , -0.2   , -0.4405, -0.    ,  0.    , -0.    ,  0.    ,
        -0.2   , -0.    ,  0.    , -0.    ,  0.    , -0.    ,  0.    ,
         0.    , -0.2   , -0.    , -0.    , -0.    ,  0.    ,  0.    ,
        -0.    , -0.    ,  0.    ,  0.    , -0.    ],
       [-0.    ,  0.    , -0.    , -0.4397, -0.    ,  0.    , -0.    ,
        -0.    ,  0.    , -0.    , -0.    , -0.    ,  0.    , -0.    ,
         0.    ,  0.    ,  0.    , -0.    ,  0.    , -0. 

In [64]:
np.round(prec_E_u,4)

array([[ 6. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  6. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  5.8,  0.2,  0. ,  0. ,  0. ,  0. ,  0. , -0.2,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.2,  5.8,  0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  6. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  6. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  6. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  5.8,  0.2,  0. ,  0. ,
         